In [1]:
import gym
import minihack
import numpy as np
from typing import List
import random
import math
import pygad
from operator import itemgetter

## Dictionary

We create a useful dictionary to define possible actions for the agent.

In [2]:
movement_dictionary = {
    0 : "north",
    1 : "east",
    2 : "south",
    3 : "west",
    4 : "north-east",
    5 : "sud-east",
    6 : "sud-west",
    7 : "north-west",
}

## Utility Functions

We define four functions that we use to extract useful information about the game environment. In particular we will then have:

- print_room : takes as input a matrix of integers and prints it in ASCII characters. It is used to visually check the state of the game.

- search_environment_indexes : takes as input a matrix of integers and returns the first index of the array other than 32. In creating the task, MiniHack gives us a 21x79 observation matrix, of which we are only interested in its submatrix of size 15x15 that represent the room. All positions outside this submatrix are passed by MiniHack with the value of 32, which in ASCII represents the empty character. This function then allows us to find the 15x15 submatrix that we will work with in the rest of the code.

- search_environment_agent_position: takes as input a matrix of integers (the one representing the room in the MiniHack task) and returns the position of the agent inside the room.

- search_environment_goal_position: takes as input a matrix of integers (the one representing the room in the MiniHack task) and returns the position of the stairs going down (our goal) inside the room.

In [3]:

def print_room(environment):
#Print the room in ASCII characters. 
    
    for row in range(len(environment[:,1])):
        for col in range(len(environment[1,:])):
            print(chr(environment[row][col]), end=' ') 
        print('\n')


def search_environment_indexes(environment):
#We search the indexes to find the 15x15 submatrix representing the MiniHack task room.

    for row in range(len(environment[:,1])):
        for col in range(len(environment[1,:])):
            if int(environment[row][col]) != 32:
                return row, col      


def search_environment_agent_position(environment: np.ndarray):
    '''
    Return agent position inside the environment.
    MiniHack passes us the position of the agent in the array with the 
    value of 64, which in ASCII turns into @.
    '''
    for row in range(len(environment[:,1])):
        for col in range(len(environment[1,:])):
            if int(environment[row][col]) == 64:  
                return row, col


def search_environment_goal_position(environment: np.ndarray):
    """ 
    Return staircase position inside the environment.
    MiniHack passes us the position of the stairs going down in the matrix 
    with the value of 62, which in ASCII turns into >.
    """
    for row in range(len(environment[:,1])):
        for col in range(len(environment[1,:])):
            if int(environment[row][col]) == 62:
                return row, col

def update_position(old_position: List[int], movement: int) -> List[int]:
    """ Return updated positions of the agent in the environment         
            old_position is a legal move
            movement: int 0..3 indication of direction
    """

    # Directions store the movements possible, if you want to move diagonally you can add more directions here
    directions = [(0, -1), (1, 0), (0, 1), (-1, 0), (-1, 1), (1, 1), (1, -1), (-1, -1)] 

    new_position = [0, 0]

    update_position = directions[movement]

    # Here we're not checking if goes out of bounds
    new_position[0] = old_position[0] + update_position[0]
    new_position[1] = old_position[1] + update_position[1]
    
    return new_position[0], new_position[1] 

def is_legal_move(old_position: List[int], movement: int) -> bool:
    """ Check if the agent is moving in a legal position """

    bounds = {
        0: (0, -1),
        1: (1, 0),
        2: (0, 1),
        3: (-1, 0),
        4: (-1, 1),
        5: (1, 1),
        6: (1, -1),
        7: (-1, -1)
        }

    new_position = old_position[0] + bounds[movement][0], old_position[1] + bounds[movement][1]
    
    return new_position[0] in range(15) and new_position[1] in range(15) 

    

# Class for Rules

We define a class for the random creation of Rules for the MiniHack task. A Rule is constructed to contain the following information:

- Position: a pair of randomly created integers indicating a position in a 15x15 matrix.

- Movement: an integer between 0 and 7 indicating the movement (following the dictionary defined earlier) that the agent must make when in the above position 

In the random creation of the Rule, we check its good definition: that is, we check that the movement associated with the position does not cause the agent to go outside the matrix.

In the class we also define the rule_list function that takes as input the agent's position seen as two integers and returns the same position seen as a single integer. 

In [4]:
class Rule:

    position: List[int] = [0, 0]
    movement: int


    def __init__(self):
       
        #Initialize a random position
        self.position[0] = random.choice(range(0, 15))
        self.position[1] = random.choice(range(0, 15))
        

        #Initialize a random movement
        self.movement = random.choice(range(0, 8))
        
        '''
        Check that the Rule is well defined. 
        after_pos_1 and after_pos_2 are the support 
        variables indicating the position in the matrix 
        after applying the movement. 
        '''
        after_pos_1=self.position[0] 
        after_pos_2=self.position[1]
        
        old_position = [after_pos_1, after_pos_2]
        
        after_pos_1, after_pos_2 = update_position(old_position, self.movement)
        
        
        #Check that the position after movement remains in the matrix    
        while after_pos_1<0 or after_pos_1>=15 or after_pos_2<0 or after_pos_2>=15:
             
            self.movement = random.choice(range(0, 7))
             
            after_pos_1=self.position[0]
            after_pos_2=self.position[1]
            
            old_position = [after_pos_1, after_pos_2]
            
            after_pos_1, after_pos_2 = update_position(old_position, self.movement)
        
    def rule_list(self):
        return [self.position[0]*15+self.position[1], self.movement]
    


# Generate MiniHack's Enivornment

We initialize the Room Random 15x15 task from MiniHack. In doing so we call in observation_keys the arrays with the information we need in the code. See the MiniHack documentation for more information.  

With env.reset() MiniHack generates a new environment (the one on which we will try to solve the task) and we save the room description matrices in the obs variable.

With env.render() MiniHack allows us to print the room, and then we see the location of the agent and the location of the staircase, where we would like the agent to go (our goal).

In [5]:
env=gym.make(
    "MiniHack-Room-Random-15x15-v0",
    observation_keys=("chars", "colors", "specials", "pixel"),
)

obs = env.reset() #Generate a new environment and save the describtions arrays in obs
env.render() #Print the room 


Hello Agent, welcome to NetHack!  You are a chaotic male human Rogue.           
                                                                                
                                                                                
                                                                                
                                ...............                                 
                                .............>.                                 
                                ...............                                 
                                ...............                                 
                                ...............                                 
                                ...............                                 
                                ...............                                 
                                ...............                                 
                           

Using the utility functions defined earlier we get the room information of the task we just created.

We note that the matrix we print in print_room is the same as the matrix above printed with MiniHack's env.render() command.

In [6]:
#find the indices where the submatrix we are interested in begins
i,j = search_environment_indexes(obs['chars'])

#print the matrix in ASCII characters. 
print("Tha matrix in ASCII that represent the random room is:")
print_room(obs['chars'][i:i+15, j:j+15])

#find the agent position and the goal position in the room    
agent_position_1, agent_position_2 = search_environment_agent_position(obs['chars'][i:i+15, j:j+15])
goal_1, goal_2 = search_environment_goal_position(obs['chars'][i:i+15, j:j+15])
    
print("The agent position is: ", agent_position_1, " ",  agent_position_2)
print("The goal position is: ", goal_1, " ", goal_2)

Tha matrix in ASCII that represent the random room is:
. . . . . . . . . . . . . . . 

. . . . . . . . . . . . . > . 

. . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . 

. . . @ . . . . . . . . . . . 

. . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . 

The agent position is:  12   3
The goal position is:  1   13


# Genetic Algorithm for Rule Creation

### Creating the Initial Population 

The population of our genetic algorithm will be a list of rules. We then initialize the empty list and fill it as we go with new randomly created rules using the Rule class and the rule_list function. 

Each gene in the population will consist of a pair of integers [x_1, x_2]:

- x_1: indicates a position in the matrix (and thus in the room) seen as a subsequence of integers (and not as indices). It is obtained, considering the position described by indices, as follows:

                                    x_1 = agent_position_1*15+agent_position_2

where agent_position_1 and agent_position_2 are the indices. 

- x_2: indicates the movement to be made when one is at position x_1. 

In [7]:
num_initial_population=1000 #numbers of genes in the initial population
list_of_rules=[]

for x in range(num_initial_population):
    
    rule=Rule()
    list_of_rules.append(rule.rule_list())
    
    #print the initial population.
    print("The", x+1, "-th rules is:")
    #print the position either as an index or as a sequence of integers.
    print("Position: (", math.floor(list_of_rules[x][0]/15), ", ", list_of_rules[x][0] % 15, ") =", list_of_rules[x][0])
    #print the movement either as an integer or as a string using dictionary
    print("Movement: ", list_of_rules[x][1], " - ", movement_dictionary[list_of_rules[x][1]])

The 1 -th rules is:
Position: ( 5 ,  0 ) = 75
Movement:  1  -  east
The 2 -th rules is:
Position: ( 10 ,  12 ) = 162
Movement:  6  -  sud-west
The 3 -th rules is:
Position: ( 4 ,  11 ) = 71
Movement:  2  -  south
The 4 -th rules is:
Position: ( 4 ,  14 ) = 74
Movement:  3  -  west
The 5 -th rules is:
Position: ( 7 ,  13 ) = 118
Movement:  2  -  south
The 6 -th rules is:
Position: ( 5 ,  13 ) = 88
Movement:  3  -  west
The 7 -th rules is:
Position: ( 11 ,  7 ) = 172
Movement:  5  -  sud-east
The 8 -th rules is:
Position: ( 6 ,  8 ) = 98
Movement:  0  -  north
The 9 -th rules is:
Position: ( 0 ,  10 ) = 10
Movement:  6  -  sud-west
The 10 -th rules is:
Position: ( 3 ,  10 ) = 55
Movement:  4  -  north-east
The 11 -th rules is:
Position: ( 10 ,  7 ) = 157
Movement:  7  -  north-west
The 12 -th rules is:
Position: ( 12 ,  7 ) = 187
Movement:  5  -  sud-east
The 13 -th rules is:
Position: ( 13 ,  9 ) = 204
Movement:  3  -  west
The 14 -th rules is:
Position: ( 4 ,  9 ) = 69
Movement:  6  - 

### Define the Fitness Function

We define the fitness function for the genetic algorithm.

The fitness function must be defined as the pygad documentation requires. The function takes as input solution, which is a gene from the genetic algorithm, and solution_idx, which is the index of the gene in the population. Specifically solution is of the form:

- solution[0]: the position in the array written as a sequence of integers

- solution[1]: the move to be made at the above position

The fitness_function must also return fitness, a numerical value expressing the goodness of the gene (in the specific case, of the rule).

In our specific case, our fitness function returns floats between 0 and 100 in the following way:

- 0 is returned in several cases. The first case is when the position of the rule coincides with the position of the goal. In fact, when the agent arrives at the goal, the task is declared resolved, and thus there is no need to possess a rule for that position. 

- 0 is also returned when the movement defined in the gene, applied to the position associated with it, results in the agent leaving the matrix. 

- If neither of the above cases happens, a numerical value is computed that takes into account two parameters. The first is whether the motion applied to the position causes the agent to approach the goal, move away from it, or remain at the same distance. This parameter is expressed in the numerator of the auxiliar variable. The second parameter takes into account the agent's distance from the goal before performing the action: the farther away the agent is, the less weight the rule will have. This parameter is expressed in the denominator of the auxiliar variable.

- The two limiting cases of the previous point are when the agent is at distance 1 from the goal before applying the movement. In this case, if the rule tells the agent to move toward the goal and complete the task, it will assume importance 100. If it moves away from the goal it will instead have importance 0. Finally, if he stays at the same distance it will have non-high importance.

In [8]:
def fitness_function(solution, solution_idx):
     
    ag_pos_before_action_1=math.floor(solution[0]/15)
    ag_pos_before_action_2=solution[0] % 15
    ag_movement=solution[1]

    #check that the generated rule is not the goal position
    if ag_pos_before_action_1==goal_1 and ag_pos_before_action_2==goal_2:
        return 0
     
    #define the new agent position after applying the action in ag_movement=solution[1]
    ag_pos_after_action_1=ag_pos_before_action_1 
    ag_pos_after_action_2=ag_pos_before_action_2

    old_position = [ag_pos_after_action_1, ag_pos_after_action_2]

    ag_pos_after_action_1, ag_pos_after_action_2 = update_position(old_position, ag_movement)
    
    #check that the new agent position after applying the ag_movement is in the matrix 
    if ag_pos_after_action_1<0 or ag_pos_after_action_1>=15 or ag_pos_after_action_2<0 or ag_pos_after_action_2>=15:
        return 0

    #calculate the distance to the goal both before applied the move and after applied the move
    distance_before=max(np.abs(ag_pos_before_action_1-goal_1), np.abs(ag_pos_before_action_2-goal_2))
    distance_after=max(np.abs(ag_pos_after_action_1-goal_1), np.abs(ag_pos_after_action_2-goal_2))
    
    #calculate the importance of the rule taking into account the distance to the goal and the goodness of movement
    if distance_after!=distance_before: 
        auxiliar=(distance_before-distance_after)/(math.sqrt(distance_before))
    else:
        auxiliar=random.randint(-1,1)/(5*math.sqrt(distance_before))
        
    fitness=50*auxiliar+50

    return fitness

### Create the Pygad Instance

We create an instance of pygad, passing the previously generated list_of_rules as the initial population, and deciding in num_gener the number of iterations of the algorithm. As parent selection we use "rws," which chooses each parent by taking it with a probability obtained by normalizing the fitness value. 

In [9]:
fitness_func = fitness_function
num_gener=10
parent_sel = "rws"
cross_type = "single_point"
    
ga_instance = pygad.GA(fitness_func=fitness_func,
                       initial_population=list_of_rules,
                       gene_type=int,
                       num_generations=num_gener,
                       num_parents_mating=num_initial_population,
                       parent_selection_type=parent_sel,
                       crossover_probability=1.0,
                       crossover_type=cross_type,
                       mutation_type= None)

ga_instance.run()

We save and print the rules generated after the last iteration of the genetic algorithm and their fitness function.

In [10]:
best_solution=ga_instance.best_solution()[0]
final_fitness_value, final_population=ga_instance.last_generation_fitness, ga_instance.last_generation_offspring_mutation

print(ga_instance.best_solution()[0])
print(final_population)
print(final_fitness_value)

[27  2]
[[ 15   2]
 [ 32   4]
 [125   4]
 ...
 [ 35   5]
 [110   5]
 [ 52   4]]
[100.          63.86750491  65.07556723  67.67766953  53.16227766
  40.          67.67766953  66.66666667  65.07556723  75.
  50.          72.36067977  66.66666667  64.43375673  75.
  64.43375673  75.          67.67766953  65.8113883   67.67766953
  65.8113883   64.43375673  67.67766953  53.77964473  68.89822365
  32.32233047  65.8113883   64.43375673  64.43375673  52.88675135
  66.66666667  64.43375673  63.86750491  64.43375673  65.8113883
  50.          63.86750491  66.66666667  63.86750491  67.67766953
  64.43375673  63.86750491  63.86750491  78.86751346  64.43375673
  66.66666667  66.66666667  67.67766953  66.66666667  75.
  67.67766953  65.8113883   50.          63.86750491  75.
  53.16227766  70.41241452  53.16227766  53.01511345  75.
 100.          67.67766953  52.77350098  53.01511345  65.07556723
  47.22649902  67.67766953  52.77350098  66.66666667  64.43375673
  75.          67.67766953  64.433756

# Generated Rules

In the following code, using the result of the genetic algorithm, we create a list containing the rules that the agent must follow to complete the task. 

There can be multiple rules for the same position in the final result of the genetic algorithm. So what we do now is to insert the rules generated by pyagd into the final_rules list, keeping in mind that: for each position there can be a maximum of one rule; and the rule we need to insert is the one with the highest fitness value. Also, if the greatest fitness value is less than 50, we don't insert the rule in the list. 

To do that, we define an auxiliary list, called auxiliar_rules, in which we insert all the rules present in the final_population with a fitness value greater or equal than 50.

We also create a matrix, called environment, that takes into account both the goal position and the positions for which a rule is defined in final_rules. In this way we can print this matrix, marking with a * the positions for which we have a rule, and visually see the distribution of the positions for which we have a rule in the room. 

In [11]:
#Create the matrix, initially with only the goal position (46 is the integer that is used to indicate the empty floor)
environment=np.full((15, 15), 46, dtype=int)
environment[goal_1][goal_2]=62

#Define an auxiliar list, with only genes with fitness greater or equal 50
ausiliar_rules=[]
ausiliar_rules.append([best_solution[0], best_solution[1], final_fitness_value[0]])

for x in range(len(final_population)) :
    if final_fitness_value[x+1] >=50:
        ausiliar_rules.append([final_population[x][0], final_population[x][1], final_fitness_value[x+1]])

#Sort the auxiliar list according to descending fitness value 
ausiliar_rules=sorted(ausiliar_rules, key=itemgetter(2), reverse=True)

#Create final_rules, initially empty
final_rules=[]

for x in range(len(ausiliar_rules)):
    axi=math.floor(ausiliar_rules[x][0]/15)
    ord=ausiliar_rules[x][0] % 15
    #If the rule_position is not already included in the final_rules, we will include it
    if environment[axi][ord]!=42:
        environment[axi][ord]=42
        final_rules.append(ausiliar_rules[x])

#Print the final rules generated        
print("The number of final rules generated is:")
print(len(final_rules))
print("Print the room, so that the positions for which a rule is defined are marked with an *:")
print_room(environment)    
print("Print the final rules that agent must use to complete the task:")
print(final_rules)

The number of final rules generated is:
61
Print the room, so that the positions for which a rule is defined are marked with an *:
. . . . . . . . . . . . . . . 

* * . * . . * . . . * . * > . 

. . * . . * . * . . . . . . . 

. . . . * . . * . . * . . . . 

* . . . . * . . . . . . . . . 

* * * . . . * * * . * * . * . 

* . . . * . . . . * . * * . . 

. * . . * * * . . . . . . * . 

* . . . . * . . . . . . . . . 

. . . . . . . . . * . * * . . 

* . . . * . * * * . . . * . . 

* * * . . . . . . . . * * . . 

* . . . . . . . * * . . * * . 

. . . . . . . . * . * . * . . 

. * . . . . * . . * . * . . . 

Print the final rules that agent must use to complete the task:
[[27, 2, 100.0], [25, 2, 78.86751345948129], [55, 2, 78.86751345948129], [85, 4, 75.0], [88, 3, 75.0], [86, 4, 75.0], [99, 3, 72.36067977499789], [101, 4, 72.36067977499789], [102, 4, 72.36067977499789], [83, 2, 72.36067977499789], [37, 5, 70.41241452319315], [118, 4, 70.41241452319315], [52, 4, 70.41241452319315], [82, 2, 

# Solving the Task 

After creating our rules through the genetic algorithm we can finally move on to solve the task. The idea for doing this is very simple. We start with the agent in the random position given by the MiniHack env.reset() command. After that, at each step, the agent performs one of the following actions:

- if a rule exists at the position it is in, it moves according to the rule's instructions.

- otherwise, it moves randomly with uniform probability. 

To check if there is a rule at the position where the agent is, I help myself with the environment matrix created earlier. If the task is completed in a low number of moves, it means that our genetic algorithm has worked well.

In [12]:
#add a counter for the number of moves of the agent.
cont=1

while agent_position_1!=goal_1 or agent_position_2!=goal_2:
        
    print("Iteration Number", cont)
    #transform the positions from indexes to sequence of integers
    ag_position=agent_position_1*15+agent_position_2
    
    #check if there is a rule in the agent position     
    if environment[agent_position_1][agent_position_2]==42:
        
        #look for the rule in the final_rules list and update agent position
        for x in range(len(final_rules)):
            if final_rules[x][0]==ag_position:

                old_position = [agent_position_1, agent_position_2]
                
                agent_position_1, agent_position_2 = update_position(old_position, final_rules[x][1])
                
                #print the information about the movement
                print("We use the rule", x, " and we move to", agent_position_1, ",", agent_position_2, end=" ")
                print("using the action", movement_dictionary[final_rules[x][1]])
                
                #move the agent with the MiniHack's command env.step()
                env.step(final_rules[x][1])
                #update the variable x to exit the for loop 
                x=10000

    #randomly move the agent if there is no rule for its position
    elif environment[agent_position_1][agent_position_2]!=42:
        
        #create random movement
        mov=random.choice(range(0, 8))   
        
        old_position = [agent_position_1, agent_position_2]
        
        agent_position_1, agent_position_2 = update_position(old_position, mov)

        #check that the random move does not come out of the matrix
        while agent_position_1<0 or agent_position_1>=15 or agent_position_2<0 or agent_position_2>=15:
            
            #create a new random movement
            agent_position_1=math.floor(ag_position/15)
            agent_position_2=ag_position % 15
            mov = random.choice(range(0, 8))
            
            old_position = [agent_position_1, agent_position_2]
        
            agent_position_1, agent_position_2 = update_position(old_position, mov)

        #move the agent with the MiniHack's command env.step()
        env.step(mov)
        #print the information about the movement
        print("We use the random movement", movement_dictionary[mov], " and we move to", agent_position_1, ",", agent_position_2)
    
    #print room to see the new agent position    
    print_room(obs['chars'][i:i+15, j:j+15])
    #print if the task is completed    
    if agent_position_1==goal_1 and agent_position_2==goal_2:
        print("Task Completed in", cont, "moves!!!!!")
    else:          
        print("###########################################################")
    #update the counter
    cont=cont+1

Iteration Number 1
We use the random movement east  and we move to 13 , 3
. . . . . . . . . . . . . . . 

. . . . . . . . . . . . . > . 

. . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . 

. . . < @ . . . . . . . . . . 

. . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . 

###########################################################
Iteration Number 2
We use the random movement north-east  and we move to 12 , 4
. . . . . . . . . . . . . . . 

. . . . . . . . . . . . . > . 

. . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . 

. . . . . . . . . 